<a href="https://colab.research.google.com/github/gabibu/nlp/blob/master/Assignment_3_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Training a simple neural named entity recognizer (NER)

In [20]:
import torch
import torch.nn as nn
import numpy as np

In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/


The three files (train, test and eval) are available from the course git repository (https://github.com/kfirbar/nlp-course)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion *read_data* for reading the data from a single file (either train, test or eval). This function recieves a filepath and returns a list of sentence. Every sentence is encoded as a pair of lists, one list contains the words and one list contains the labels.

In [21]:
!git clone https://github.com/kfirbar/nlp-course

Cloning into 'nlp-course'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 58 (delta 24), reused 31 (delta 8), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [22]:
def read_data(filepath):
  data = []
  with open(filepath) as file_in:
    current_line_words = []
    current_line_tags = []
    for line in file_in:

      if line == '\n':
        data.append((current_line_words, current_line_tags))
        current_line_words = []
        current_line_tags = []
      else:
        word, tag = line.strip().split(' ')
        current_line_words.append(word)
        current_line_tags.append(tag)

    if any(current_line_words):
        data.append((current_line_words, current_line_tags))

    return data


train = read_data('/content/nlp-course/connl03_train.txt')
test = read_data('/content/nlp-course/connl03_test.txt')
dev = read_data('/content/nlp-course/connl03_dev.txt')
len(train), len(dev), len(test)

(1750, 250, 500)

In [4]:
!ls nlp-course

connl03_dev.txt   connl03_train.txt  lm-languages-data-new
connl03_test.txt  lm-languages-data  README.md


The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [23]:
UNK_TOKEN = 0

class Vocab:
    def __init__(self):
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
        
    def index_words(self, words):
      word_indexes = [self.index_word(w) for w in words]
      return word_indexes

    def index_tags(self, tags):
      tag_indexes = [self.tag2id[t] for t in tags]
      return tag_indexes
    
    def index_word(self, w):
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        return self.word2id[w]
            

**Task 2:** Write a function *prepare_data* that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,labels) to a pair of indexes (from Vocab). Each pair should be added to *data_sequences*, which is returned back from the function.

In [24]:
vocab = Vocab()

for (words, _) in train:
  for word in words:
    vocab.index_word(word)

In [26]:
def prepare_data(data, vocab):
    data_sequences = []
    
    for (words, tags) in data:
      words_ids = vocab.index_words(words)
      tags_ids = vocab.index_tags(tags)
      data_sequences.append((words_ids, tags_ids))

      

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

full_eval  = list(dev_sequences)
full_eval.extend(test_sequences)
len(train_sequences), len(dev_sequences), len(test_sequences), len(full_eval)

(1750, 250, 500, 750)

In [8]:
train_sequences[0:1], train[0:1]


([([1, 2, 3, 4], [5, 0, 5, 0])],
 [(['Portuguesa', '2', 'Parana', '0'], ['B-ORG', 'O', 'B-ORG', 'O'])])

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

In [34]:
class NERNet(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions, pretrained_embeddings = None):
        super(NERNet, self).__init__()
        # TODO: your code...
        self.embedding = nn.Embedding(input_size, embedding_size)
        if pretrained_embeddings is not None:
          self.embedding.weight = nn.Parameter(pretrained_embeddings)

        bidirectional = True if directions==2 else False

        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(bidirectional))
        self.out = nn.Linear(hidden_size * 2 if bidirectional else hidden_size , output_size)
    
    def forward(self, input_sentence):

        input = input_sentence.reshape(-1, input_sentence.shape[0])
        embeddings = self.embedding(input)

        lstm_out, _ = self.lstm(embeddings)

        output = self.out(lstm_out)

        return output

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [28]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [29]:
def train_loop(model, n_epochs, trainning_data, print_loss = False ):
  # Loss function

  criterion = nn.CrossEntropyLoss()

  # Optimizer (ADAM is a fancy version of SGD)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  
  for e in range(1, n_epochs + 1):
    
    epoc_loss = 0.0 
    for (words_ids, tags_ids) in trainning_data:
      words_ids = torch.from_numpy(np.array(words_ids)).to(device)
      target = torch.from_numpy(np.array(tags_ids)).reshape(1, len(tags_ids)).to(device)
      res = model(words_ids)
      res = torch.transpose(res, 1, 2)
      loss = criterion(res, target)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoc_loss += loss.item()
    
    if print_loss:
      print('epoc {} loss {}'.format(e, epoc_loss))

# model = NERNet(len(vocab.id2word), 300, 500, len(vocab.tag2id), 1, 1).to(device)
# train_loop(model, 10, train_sequences)

**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [30]:
def evaluate(model, caption):
  model.eval()

  labels = []
  predicted = []

  with torch.no_grad():

    for (words_ids, tags_ids) in full_eval:
      words_ids = torch.from_numpy(np.array(words_ids)).to(device)
      #target = np.array(tags_ids)
      labels.extend(tags_ids)
      res = model(words_ids)
      probs = torch.softmax(res, dim=2)
      predicted_labels = torch.argmax(probs, dim=2).cpu().detach().numpy().flatten().tolist()

      predicted.extend(predicted_labels)


  from sklearn.metrics import recall_score
  from sklearn.metrics import precision_recall_fscore_support as score
  import pandas as pd
  precision, recall, _, _ = score(labels, predicted)

  recall  = recall.tolist()
  precision  = precision.tolist()
  
  col_id_and_tag = sorted([(id, tag) for (id, tag) in vocab.id2tag.items()], key = lambda x: x[0])
  cols_names = [x[1] for x in col_id_and_tag]
  cols_ids = [x[0] for x in col_id_and_tag]
  global_precision, global_recall, _, _ = score(labels, predicted, labels = cols_ids[1:], average='micro')


  recall.append(global_recall)
  precision.append(global_precision)

  datas = []
  datas.append(recall)
  datas.append(precision)

  print('\n')
  print(caption)
  df = pd.DataFrame(datas, columns = cols_names + ['Global'], index =['recall', 'precision'])
  print(df.to_string(index=False))



# evaluate(model, "test")


**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [37]:
eval_model_hyperparamaters = [(500, 1, 1), (500,2, 1), (500,3,1), (500,1, 2), (500,2, 2), (500,3, 2), (800, 1,2 ), (800, 2, 2), (800, 3, 2)]

In [50]:
# TODO - your code goes here...

NUM_OF_EPOCS = 10 

for (model_index, (hidden_size, n_layers, directions)) in enumerate(eval_model_hyperparamaters):
  
  current_model = NERNet(len(vocab.id2word), 300, hidden_size, len(vocab.tag2id), n_layers, directions).to(device)
  caption = 'Model {}: (hidden_size: {}, n_layers: {}, directions: {})'.format(model_index+1, hidden_size, n_layers, directions)
  train_loop(current_model, NUM_OF_EPOCS, train_sequences)

  evaluate(current_model, caption)


Model 1: (hidden_size: 500, n_layers: 1, directions: 1)
        O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG    Global
 0.968436  0.544164  0.401766  0.676806  0.407895  0.538610  0.332278  0.514467
 0.928005  0.658397  0.472727  0.816514  0.939394  0.535509  0.517241  0.617507
Model 2: (hidden_size: 500, n_layers: 2, directions: 1)
        O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG    Global
 0.957156  0.599369  0.432671  0.730038  0.473684  0.573359  0.424051  0.565597
 0.937367  0.645161  0.424242  0.791753  0.923077  0.621339  0.503759  0.615351
Model 3: (hidden_size: 500, n_layers: 3, directions: 1)
        O     B-PER     I-PER     B-LOC     I-LOC     B-ORG     I-ORG    Global
 0.946186  0.596215  0.428256  0.718631  0.394737  0.544402  0.373418  0.546968
 0.940831  0.530899  0.303125  0.849438  0.833333  0.699752  0.508621  0.559157
Model 4: (hidden_size: 500, n_layers: 1, directions: 2)
        O     B-PER     I-PER     B-LOC     I-LOC     B-

**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-06-07 17:46:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-07 17:46:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-07 17:46:40--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [4]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [5]:
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data


In [33]:
def load_glove_embeddings(path, word2idx, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(word2idx), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            if word not in word2idx:
              continue

            index = word2idx[word]
            vector = np.array(values[1:], dtype='float32')
            embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

glove_fname = '/content/glove.6B.300d.txt'

glove_embeddings = load_glove_embeddings(glove_fname, vocab.word2id, 300)
glove_embeddings.shape, len(vocab.word2id)

(torch.Size([8955, 300]), 8955)

In [ ]:
# TODO - your code goes here...

NUM_OF_EPOCS = 10 

for (model_index, (hidden_size, n_layers, directions)) in enumerate(eval_model_hyperparamaters):
  
  current_model = NERNet(len(vocab.id2word), 300, hidden_size, len(vocab.tag2id), n_layers, directions, glove_embeddings).to(device)
  caption = 'Model {}: (hidden_size: {}, n_layers: {}, directions: {})'.format(model_index+1, hidden_size, n_layers, directions)
  train_loop(current_model, NUM_OF_EPOCS, train_sequences)

  evaluate(current_model, caption)

**Good luck!**